#  Машинный перевод с использованием рекуррентных нейронных сетей

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Deep Learning with PyTorch (2020) Авторы: Eli Stevens, Luca Antiga, Thomas Viehmann
* https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html#the-seq2seq-model
* https://www.adeveloperdiary.com/data-science/deep-learning/nlp/machine-translation-recurrent-neural-network-pytorch/

## Задачи для совместного разбора

1\. Рассмотрите пример архитектуры Encoder-Decoder с использованием RNN. Обсудите концепцию teacher forcing.

In [1]:
import torch as th
import torch.nn as nn

In [32]:
n_ru_tokens = 1000
batch_size = 16
ru_seq_len = 15
n_en_tokens = 500
en_seq_len = 10

ru = th.randint(0, n_ru_tokens, size=(batch_size, ru_seq_len))
en = th.randint(0, n_en_tokens, size=(batch_size, en_seq_len))

In [ ]:
class Encoder(nn.Module):
  def __init__(self, embedding_dim, hidden_size):
      super().__init__()
      self.emb = nn.Embedding(
          num_embeddings=n_ru_tokens,
          embedding_dim=embedding_dim,
          padding_idx=0
      )
      self.dropout = nn.Dropout(p=0.5)
      self.rnn = nn.GRU(embedding_dim, hidden_size, batch_first=True)

  def forward(self, X):
    out = self.emb(X) # batch x seq x emb_size
    out = self.dropout(out)
    _, h = self.rnn(out) # out: batch x seq x hidden_size
    return h # 1 x batch x hidden_size

In [ ]:
embedding_dim = 100
encoder_hidden_size = 300

encoder = Encoder(embedding_dim, encoder_hidden_size)

encoder_output = encoder(ru)

In [ ]:
encoder_output.shape

torch.Size([1, 16, 300])

In [ ]:
class Decoder(nn.Module):
  def __init__(self, embedding_dim, decoder_hidden_size):
    super().__init__()
    self.emb = nn.Embedding(
          num_embeddings=n_en_tokens,
          embedding_dim=embedding_dim,
          padding_idx=0
    )
    self.rnn = nn.GRUCell(embedding_dim, decoder_hidden_size)
    self.fc = nn.Linear(decoder_hidden_size, n_en_tokens)

  def forward(self, encoder_output, labels):
    # labels: batch x seq_len - считаем, что в 0 столбце SOS
    # encoder_output: 1 x batch x encoder_hidden_size
    seq_len = labels.size(1)
    input_tokens = labels[:, 0]
    decoder_hidden = encoder_output[0]
    for _ in range(1, seq_len):
      out = self.emb(input_tokens).relu() # batch x emb_size
      decoder_hidden = self.rnn(out, decoder_hidden) # batch x dec_hidden
      out = self.fc(decoder_hidden) # batch x n_en_tokens

      # teacher forcing
      input_tokens = out.argmax(dim=1).detach()
      # ...


    # вернуть прогнозы для каждого эл-та последовательности
    # batch x seq x n_en_token
    return ...

In [ ]:
decoder = Decoder(embedding_dim=100, decoder_hidden_size=300)
decoder(encoder_output, en)

Ellipsis

In [ ]:
class Decoder(nn.Module):
    def __init__(self, emb_dim, hidden_size, n_ru):
        super().init()
        self.emb = nn.Embedding(num_embeddings=n_en_tokens, embedding_dim=emb_dim, padding_idx=0)
        nn.emb = GRUCell(embedding_dim, hidden_size) # gru mojno
        nn.fc1 = nn.Linear()

    def forward(self, encoder_output, labels): # правильные ответы на языке на который переводим
        # labels: batch x seq_len
        # encoder_output: 1 x batch x encoder_hidden_size # считаем что в нулвом столбце - сос
        input_tokens = labels[:,0]
        decoder_hidden = encoder_output[0]

        seq_len = labels.size(1)

        for _ in range(1, seq_len):
            out = self.emb(input_tokens).relu() # batch_size x emb_dim
            decoder_hidden = self.rnn(out, decoder_hidden) # batch x decoder_hidden
            out = self.fc(decoder_hidden) # batch x n_en_tokens

            input_tokens = out.argmax(dim=1).detach()

        return # все сохраненные ауты

## Задачи для самостоятельного решения

<p class="task" id="1"></p>

1\. Считайте файлы `RuBQ_2.0_train.json` (обучающее множество) и `RuBQ_2.0_test.json` (тестовое множество). Для каждого файла создайте по списка: список предложений на русском языке и список предложений на английском языке. Выведите на экран количество примеров в обучающей и тестовой выборке.

- [ ] Проверено на семинаре

In [2]:
import pandas as pd

train_df = pd.read_json('RuBQ_2.0_train.json')
test_df = pd.read_json('RuBQ_2.0_test.json')

In [3]:
train_df.head(3)

,uid,question_text,query,answer_text,question_uris,question_props,answers,paragraphs_uids,tags,RuBQ_version,question_eng
0,0,Что может вызвать цунами?,SELECT ?answer \nWHERE {\n wd:Q8070 wdt:P828 ...,Землетрясение,[http://www.wikidata.org/entity/Q8070],[wdt:P828],"[{'type': 'uri', 'label': 'землетрясение', 'va...","{'with_answer': [35622], 'all_related': [35622...",[1-hop],1,What can cause a tsunami?
1,1,Кто написал роман «Хижина дяди Тома»?,SELECT ?answer \nWHERE {\n wd:Q2222 wdt:P50 ?...,Г. Бичер-Стоу,[http://www.wikidata.org/entity/Q2222],[wdt:P50],"[{'type': 'uri', 'label': 'Гарриет Бичер-Стоу'...","{'with_answer': [35652], 'all_related': [35652...",[1-hop],1,"Who wrote the novel ""uncle Tom's Cabin""?"
2,2,Кто автор пьесы «Ромео и Джульетта»?,SELECT ?answer \nWHERE {\n wd:Q83186 wdt:P50 ...,Шекспир,[http://www.wikidata.org/entity/Q83186],[wdt:P50],"[{'type': 'uri', 'label': 'Уильям Шекспир', 'v...","{'with_answer': [35676, 35677], 'all_related':...",[1-hop],1,"Who is the author of the play ""Romeo and Juliet""?"


In [4]:
len(train_df)

2330

In [5]:
rus_sents = train_df['question_text'].values.tolist()
en_sents = train_df['question_eng'].values.tolist()

rus_sents_test = test_df['question_text'].values.tolist()
en_sents_test = test_df['question_eng'].values.tolist()
len(rus_sents), len(rus_sents_test)

(2330, 580)

<p class="task" id="2"></p>

2\. Создайте два Vocab на основе загруженных данных: `ru_vocab` для слов на русском языке и `en_vocab` для слов на английском языке (словари создаются на основе обучающего множества). Добавьте в словари специальные токены `<PAD>`, `<SOS>`, `<EOS>`. Выведите на экран количество токенов в полученных словарях. Выведите на экран максимальное кол-во слов в предложениях на русском языке и в предложениях на английском языке (в обучающей выборке).

- [ ] Проверено на семинаре

In [6]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [7]:
import re
from nltk.tokenize import word_tokenize

def get_corpus(sents):
    corpus = []
    for sent in sents:
      sent = re.sub(r'[^\w\s]+', '', sent.lower().strip())
      words = word_tokenize(sent)
      corpus.append(words)
    return corpus

corpus_ru = get_corpus(rus_sents)
corpus_en = get_corpus(en_sents)
corpus_ru_test = get_corpus(rus_sents_test)
corpus_en_test = get_corpus(en_sents_test)

In [8]:

from torchtext.vocab import build_vocab_from_iterator

ru_vocab = build_vocab_from_iterator(corpus_ru, specials=['<pad>', '<unk>', '<sos>', '<eos>'])
ru_vocab.set_default_index(ru_vocab['<unk>'])
en_vocab = build_vocab_from_iterator(corpus_en, specials=['<pad>', '<unk>', '<sos>', '<eos>'])
en_vocab.set_default_index(en_vocab['<unk>'])

In [9]:
len(ru_vocab.get_stoi()), len(en_vocab.get_stoi())

(5923, 4333)

In [10]:
max([len(sent) for sent in corpus_ru]), max([len(sent) for sent in corpus_en])

(25, 31)

<p class="task" id="3"></p>

3\. Создайте класс `RuEnDataset`. Реализуйте `__getitem__` таким образом, чтобы он возвращал кортеж `(x, y)`, где x - это набор индексов токенов для предложений на русском языке, а `y` - набор индексов токенов для предложений на английском языке. Используя преобразования, сделайте длины наборов индексов одинаковой фиксированной длины, добавьте в начало каждого набора индекс `<SOS>`, а в конец - индекс токена `<EOS>`. Создайте датасет для обучающей и тестовой выборки.

- [ ] Проверено на семинаре

In [11]:
from torch.utils.data import Dataset
import torchtext.transforms as T

class RuEnDataset(Dataset):
    def __init__(self, corpus_ru, corpus_en, ru_vocab, en_vocab):
        self.corpus_ru = corpus_ru
        self.corpus_en = corpus_en
        self.ru_vocab = ru_vocab
        self.en_vocab = en_vocab
        self.trans_ru = T.Sequential(
            T.AddToken('<sos>', begin=True),
            T.AddToken('<eos>', begin=False),
            T.VocabTransform(ru_vocab),
            T.ToTensor(0),
            T.PadTransform(max_length=self.get_max_len(), pad_value=0)
        )
        self.trans_en = T.Sequential(
            T.AddToken('<sos>', begin=True),
            T.AddToken('<eos>', begin=False),
            T.VocabTransform(en_vocab),
            T.ToTensor(0),
            T.PadTransform(max_length=self.get_max_len() + 2, pad_value=0)
        )

    def __getitem__(self, idx):
        a = self.corpus_ru[idx]
        b = self.corpus_en[idx]
        return self.trans_ru(a), self.trans_en(b)

    def __len__(self):
        return len(self.corpus_en)

    def get_max_len(self):
        return max(len(y) for y in self.corpus_en)

In [12]:
ru_en_dset = RuEnDataset(corpus_ru, corpus_en, ru_vocab, en_vocab)
ru_en_dset[0]

(tensor([   2,   27,  688, 2304, 5691,    3,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0]),
 tensor([   2,    6,  148,  236,   22, 4080,    3,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0]))

In [13]:
train_dset = RuEnDataset(corpus_ru, corpus_en, ru_vocab, en_vocab)
test_dset = RuEnDataset(corpus_ru_test, corpus_en_test, ru_vocab, en_vocab)

<p class="task" id="4"></p>

4\. Опишите модель `Encoder`, которая возвращает скрытое состояние рекуррентного слоя в соотстветствии со следующей схемой. Пропустите через эту модель первые 16 предложений на русском языке и выведите размер полученного результата на экран. Результатом должен являться тензор размера `1 x batch_size x hidden_dim` (если используется один однонаправленный рекуррентный слой и `batch_first=True`).

* количество эмбеддингов равно количеству слов на русском языке;
* размерность эмбеддингов выберите самостоятельно;
* при создании слоя эмбеддингов укажите `padding_idx`;
* размер скрытого состояния рекуррентного слоя выберите самостоятельно.

![encoder](https://i0.wp.com/www.adeveloperdiary.com/wp-content/uploads/2020/10/Machine-Translation-using-Recurrent-Neural-Network-and-PyTorch-adeveloperdiary.com-1.png?w=815&ssl=1)

- [ ] Проверено на семинаре

In [14]:
class Encoder(nn.Module):
  def __init__(self, n_ru_tokens, embedding_dim, hidden_size):
      super().__init__()
      self.emb = nn.Embedding(
          num_embeddings=n_ru_tokens,
          embedding_dim=embedding_dim,
          padding_idx=0
      )
      self.dropout = nn.Dropout(p=0.5)
      self.rnn = nn.GRU(embedding_dim, hidden_size, batch_first=True)

  def forward(self, X):
    out = self.emb(X) # batch x seq x emb_size
    out = self.dropout(out)
    _, h = self.rnn(out) # out: batch x seq x hidden_size
    return h # 1 x batch x hidden_size

In [15]:
from torch.utils.data import DataLoader
dl = DataLoader(train_dset, batch_size=16)
n_ru_tokens = len(ru_vocab.get_stoi())
embedding_dim = 350
hidden_size = 200

encoder = Encoder(n_ru_tokens, embedding_dim, hidden_size)
for X, y in dl:
    h = encoder(X)
    break
h.shape

torch.Size([1, 16, 200])

<p class="task" id="5"></p>

5\. Опишите модель `Decoder`, которая возвращает прогноз (набор индексов слов на английском языке). Пропустите через эту модель тензор скрытых состояний кодировщика, полученный в предыдущей задачи, и выведите размер полученного результата на экран. Результатом должен являться тензор размера `batch_size x seq_len x n_en_words` (если используется один однонаправленный рекуррентный слой и `batch_first=True`).

* количество эмбеддингов равно количеству слов на английском языке;
* размер выходного слоя равен количеству слов на английском языке;
* размерность эмбеддингов выберите самостоятельно;
* при создании слоя эмбеддингов укажите `padding_idx`;
* размер скрытого состояния рекуррентного слоя выберите самостоятельно.

![decoder](https://i2.wp.com/www.adeveloperdiary.com/wp-content/uploads/2020/10/Machine-Translation-using-Recurrent-Neural-Network-and-PyTorch-adeveloperdiary.com-2.png?w=899&ssl=1)

- [ ] Проверено на семинаре

In [16]:
class Decoder(nn.Module):
    def __init__(self, n_en_tokens, embedding_dim, decoder_hidden_size):
        super().__init__()
        self.n_en_tokens = n_en_tokens
        self.emb = nn.Embedding(
              num_embeddings=n_en_tokens,
              embedding_dim=embedding_dim,
              padding_idx=0
        )
        self.rnn = nn.GRU(embedding_dim, decoder_hidden_size, batch_first=True)
        self.fc = nn.Linear(decoder_hidden_size, n_en_tokens)

    def forward(self, encoder_output, labels): # правильные ответы на языке на который переводим
        # labels: batch_size x seq_len
        # encoder_output: 1 x batch x encoder_hidden_size # считаем что в нулвом столбце - сос
        input_tokens = labels[:,0]
        decoder_hidden = encoder_output[0]

        target_len = labels.size(0)
        seq_len = labels.size(1)
        target_vocab_size = self.n_en_tokens
        #batch_size =
        predictions = th.zeros(target_len, seq_len, target_vocab_size)
        for i in range(1, seq_len):
            out = self.emb(input_tokens).relu() # batch_size x emb_dim
            decoder_hidden, h = self.rnn(out) # batch x decoder_hidden
            out = self.fc(decoder_hidden) # batch x n_en_tokens

            #input_tokens = out.argmax(dim=1).detach()
            predictions[:,i - 1] = out
            input_tokens = labels[:, i]

        return predictions # все сохраненные ауты

In [17]:
embedding_dim = 350
decoder_hidden_size = 200
n_en_tokens = len(en_vocab.get_stoi())

decoder = Decoder(n_en_tokens, embedding_dim, decoder_hidden_size)

for X, y in dl:
    pred = decoder(h, y)
    break
pred.shape

torch.Size([16, 33, 4333])

<p class="task" id="6"></p>

6\. Объедините модели `Encoder` и `Decoder` в одну модель `EncoderDecoder`. Пропустите через эту модель первые 16 предложений на русском языке и выведите размер полученного результата на экран. Сделайте полученный результат двумерным, объединив две первые размерности: `batch_size * seq_len x n_en_words`. Выведите размерность полученного результата на экран.

- [ ] Проверено на семинаре

In [18]:
class EncoderDecoder(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, X, y):

        hidden = self.encoder(X)
        outputs = self.decoder(hidden, y)

        return outputs

In [20]:
decoder = Decoder(n_en_tokens, embedding_dim, decoder_hidden_size)
encoder_decoder = EncoderDecoder(encoder, decoder)
all = []

for X_, y_ in dl:
    res = encoder_decoder.forward(X_, y_)
    break
res.shape

torch.Size([16, 33, 4333])

In [21]:
res.reshape(-1, res.size(2)).shape

torch.Size([528, 4333])

<p class="task" id="7"></p>

7\. Настройте модель, решив задачу классификации на основе прогнозов модели `EncoderDecoder`. Игнорируйте токен `<PAD>` при расчете ошибки. Во время обучения выводите на экран значения функции потерь для эпохи (на обучающем множестве), значение accuracy по токенам (на обучающем множестве) и пример перевода, сгенерированного моделью. После завершения обучения посчитайте BLEU для тестового множества.

- [ ] Проверено на семинаре

In [22]:
!pip install torchmetrics

In [23]:
import torch.optim as optim
import torchmetrics

In [25]:
n_epochs = 7
lr = 0.01
device = 'cuda' if th.cuda.is_available() else 'cpu'
decoder = Decoder(n_en_tokens, embedding_dim, decoder_hidden_size)
model = EncoderDecoder(encoder, decoder).to(device=device)
crit = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(model.parameters(), lr=lr)
all = []


for epoch in range(n_epochs):
    losses = []
    acc_m = torchmetrics.Accuracy(task='multiclass', num_classes=len(en_vocab.get_stoi()), ignore_index=0)
    for X, y in dl:
        out = model.forward(X.to(device=device), y.to(device=device))
        all.append(out.argmax(dim=2))
        loss = crit(out.reshape(-1, out.size(2)), y.view(-1))
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        losses.append(loss)
        acc_m.update(
            out.reshape(-1, out.size(2)).argmax(dim=1).to(device='cpu'),
            y.view(-1)
        )

    print('epoch= ', epoch)
    print('loss= ', th.tensor(losses).mean().item())
    check = out.argmax(dim=2)[0].reshape(-1).tolist()
    ans = en_vocab.lookup_tokens(check)
    print(' '.join(ans))
    print('accuracy= ', acc_m.compute().item())
    print()

epoch=  0
loss=  2.8213284015655518
<sos> how tall was live freezing <eos> c c c c c c c c c c c c c c c c c c c c c c c c c <pad>
accuracy=  0.6868424415588379

epoch=  1
loss=  1.4861609935760498
<sos> how tall was participate mars <eos> c c c c c c c c c c c c c c c c c c c c c c c c c <pad>
accuracy=  0.7993064522743225

epoch=  2
loss=  1.0277494192123413
<sos> how tall was objects c <eos> c c c c c c c c c c c c c c c c c c c c c c c c c <pad>
accuracy=  0.8367025256156921

epoch=  3
loss=  0.6518498659133911
<sos> how tall was bush senior <eos> sources sources sources sources sources sources sources sources sources sources sources sources sources sources sources sources sources sources sources sources sources sources sources sources sources <pad>
accuracy=  0.8843047022819519

epoch=  4
loss=  0.30423668026924133
<sos> how tall was bush senior <eos> equator equator equator equator equator equator equator equator equator equator equator equator equator equator equator equator equ

In [32]:
dl_test = DataLoader(test_dset, batch_size=16)

In [49]:
translated, tr_y = [], []
for X, y in dl_test:
    out = model.forward(X.to(device=device), y.to(device=device))
    out = out.argmax(dim=2)
    for sent in out:
        translated.append(en_vocab.lookup_tokens(sent.tolist()))
    for Y in y:
        tr_y.append(en_vocab.lookup_tokens(Y.tolist()))

In [80]:
trs, trs_y = [], []
for lst in translated:
    if "<eos>" in lst:
        index = lst.index("<eos>")
    trs.append(lst[1:index])

for lst in tr_y:
    if "<eos>" in lst:
        index = lst.index("<eos>")
    trs_y.append(lst[1:index])

In [86]:
ref = []
for sent in corpus_en_test:
    ref.append([sent])

In [87]:
from torchtext.data.metrics import bleu_score

candidate_corpus = trs
reference_corpus = ref
bleu_score(candidate_corpus, reference_corpus)

0.6965598294299764

## Обратная связь
- [x] Хочу получить обратную связь по решению